In [1]:
import util

import jax
import jax.numpy as np

import pandas as pd

import matplotlib.pyplot as plt


Bad key "nbagg.transparent" on line 426 in
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.2.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key "animation.mencoder_path" on line 509 in
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.2.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key "animation.mencoder_args" on line 512 in
/usr/local/Cellar/python3/3.6.4_2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle.
You probably need to get an update

In [12]:
import numpy as base_np
from epiweeks import Week, Year
start = '2020-03-15'
forecast_start = '2020-04-19'

num_weeks = 8
data = util.load_state_data()
places = sorted(list(data.keys()))
#places = ['AK', 'AL']

allQuantiles = [0.01,0.025]+list(np.arange(0.05,0.95+0.05,0.05)) + [0.975,0.99]

forecast_date = pd.to_datetime('2020-04-19')
currentEpiWeek = Week.fromdate(forecast_date) - 1


forecast = {'quantile':[], 'value':[], 'type':[], 'location':[], 'target':[]}

print (currentEpiWeek)
for place in places:
    prior_samples, mcmc_samples, post_pred_samples = util.load_samples(place, path='out')
    forecast_samples = post_pred_samples['z_future']
    t = pd.date_range(start=forecast_start, periods=forecast_samples.shape[1], freq='D')
    weekly_df = pd.DataFrame(index=t, data=np.transpose(forecast_samples)).resample("1w",label='right').last()
    weekly_df[weekly_df<0.] = 0.
    for time, samples in weekly_df.iterrows():
        for q in allQuantiles:
                 deathPrediction = base_np.percentile(samples,q*100)
                 forecast["quantile"].append("{:.3f}".format(q))
                 forecast["value"].append(deathPrediction)
                 forecast["type"].append("quantile")
                 forecast["location"].append(place)
                 horizon_date = Week.fromdate(time)
                 week_ahead = horizon_date.week - currentEpiWeek.week
                 forecast["target"].append("{:d} wk ahead cum death".format(week_ahead))
                 currentEpiWeek_datetime = currentEpiWeek.startdate()
                 forecast["forecast_date"] = "{:4d}-{:02d}-{:02d}".format(currentEpiWeek_datetime.year,currentEpiWeek_datetime.month,currentEpiWeek_datetime.day)
                 if q==0.50:
                     forecast["quantile"].append("NA")
                     forecast["value"].append(deathPrediction)
                     forecast["type"].append("point")
                     forecast["location"].append(place)
                     forecast["target"].append("{:d} wk ahead cum death".format(week_ahead))
                     forecast["forecast_date"] = "{:4d}-{:02d}-{:02d}".format(currentEpiWeek_datetime.year,currentEpiWeek_datetime.month,currentEpiWeek_datetime.day)
    #base_np.quantile(hosp,axis=1,q=allQuantiles)

202016


In [13]:

forecast = pd.DataFrame(forecast)


In [14]:
forecast.loc[forecast.type=="point"]


,quantile,value,type,location,target,forecast_date
12,NA,13.168737,point,AK,1 wk ahead cum death,2020-04-12
36,NA,16.890155,point,AK,2 wk ahead cum death,2020-04-12
60,NA,19.248852,point,AK,3 wk ahead cum death,2020-04-12
84,NA,21.025867,point,AK,4 wk ahead cum death,2020-04-12
108,NA,22.265180,point,AK,5 wk ahead cum death,2020-04-12
...,...,...,...,...,...,...
32940,NA,32.377968,point,WY,23 wk ahead cum death,2020-04-12
32964,NA,33.657459,point,WY,24 wk ahead cum death,2020-04-12
32988,NA,33.796215,point,WY,25 wk ahead cum death,2020-04-12
33012,NA,34.572313,point,WY,26 wk ahead cum death,2020-04-12


In [15]:

fips_codes = pd.read_csv('/Users/gcgibson/covid19-forecast-hub/template/state_fips_codes.csv')


In [16]:
df_truth = forecast.merge(fips_codes, left_on='location', right_on='state', how='left')
df_truth["state_code"] = df_truth["state_code"].astype(int)
df_truth = df_truth[["quantile", "value", "type", "state_code","target","forecast_date"]]

df_truth = df_truth.rename(columns={"state_code": "location"})



In [17]:
import datetime

df_truth['location'] = df_truth['location'].apply(lambda x: '{0:0>2}'.format(x))
#df_truth['forecast_date'] = datetime.datetime(2020, 4, 19)
df_truth.to_csv(f'out/sub.csv', float_format="%.0f")
df_truth

,quantile,value,type,location,target,forecast_date
0,0.010,6.440057,quantile,02,1 wk ahead cum death,2020-04-12
1,0.025,7.684174,quantile,02,1 wk ahead cum death,2020-04-12
2,0.050,8.549468,quantile,02,1 wk ahead cum death,2020-04-12
3,0.100,9.563435,quantile,02,1 wk ahead cum death,2020-04-12
4,0.150,10.106796,quantile,02,1 wk ahead cum death,2020-04-12
...,...,...,...,...,...,...
33043,0.850,55.615536,quantile,56,27 wk ahead cum death,2020-04-12
33044,0.900,62.090803,quantile,56,27 wk ahead cum death,2020-04-12
33045,0.950,74.131952,quantile,56,27 wk ahead cum death,2020-04-12
33046,0.975,88.032991,quantile,56,27 wk ahead cum death,2020-04-12
